<a href="https://colab.research.google.com/github/kopaprin/takeout_yokohama/blob/master/TakeoutYokohama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/work/200507_TakeOutYokohama")
!pip install googlemaps


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Created wheel for googlemaps: filename=googlemaps-4.3.1-cp36-none-any.whl size=37711 sha256=d0bcde6bbc31c4375f6deff80a823a57d05e49913976dadb1e9db9c06ec878b9
  Stored in directory: /root/.cache/pip/wheels/85/13/9a/6be797f54ea8759a77961114dcbac86fef2a6cc7a4d6ad9785
Successfully built googlemaps


In [0]:
import pandas as pd
base_df = pd.read_csv("takeoutyokohama.csv")

In [0]:
col_name = [
            "timestamp"
            , "shinki_kubun"
            , "tenpo_mei"
            , "address_ku"
            , "address_tyou"
            , "address_tatemono"
            , "ido_keido"
            , "tell"
            , "eki"
            , "jyoken"
            , "teikyubi"
            , "url"
            , "sns"
            , "ryouri"
            , "takeout"
            , "delivery"
            , "delivery_fee"
            , "message"
            , "menu_01"
            , "menu_01_price"
            , "menu_02"
            , "menu_02_price"
            , "menu_03"
            , "menu_03_price"
            , "menu_04"
            , "unnamed"
]

In [0]:
def cleaning_base_df(df):
  result = df.copy()
  result.columns = col_name
  
  # drop duplicated
  result = result[~result.duplicated(keep='last',subset="tenpo_mei")]
  print(df.shape, "->", result.shape)

  # make address
  result["address"] = "神奈川県横浜市" + result["address_ku"] + result["address_tyou"]

  result = result.reset_index(drop=True)

  return result

result_df = cleaning_base_df(base_df)

(505, 26) -> (460, 26)


In [0]:
def add_locate(df,address):
  import googlemaps
  from fastprogress.fastprogress import  progress_bar
  googleapikey = 'hogehogehogehogehoge'
  gmaps = googlemaps.Client(key=googleapikey)

  address_list = list(df[address])
 
  lat_list = []
  lng_list = []

  for ad in progress_bar(address_list):
    gmap_list = gmaps.geocode(ad)
    try:
      lat_list.append(gmap_list[0]["geometry"]["location"]["lat"])
      lng_list.append(gmap_list[0]["geometry"]["location"]["lng"])
    except :
      lat_list.append("")
      lng_list.append("")

  result = df.copy()
  result["latitude"] = lat_list
  result["longitude"] = lng_list

  return result

add_locate_df = add_locate(result_df,"address")



In [0]:
add_locate_df.to_csv("takeoutyokohama_result_df.csv")